In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from multiprocessing import Pool, Lock, Value
from multiprocessing.dummy import Pool as ThreadPool
from time import sleep
import requests
import pandas as pd

In [2]:
authors_id = {
  "Дарья Донцова":  29369,
  "Джеймс Роллинс": 29442,
  "Макс Фрай":      102994,
  "Эрин Хантер":    26149,
  "Дмитрий Емец":   35952
}

In [3]:
driver = webdriver.Chrome('C:/Users/miracle/Jupyter/Homework/Parsing/chromedriver.exe')
books_id = []
for name in authors_id:
    books_url = "https://www.bookvoed.ru/author/books?id=" + str(authors_id[name])
    driver.get(books_url)
    sleep(1)
    notEnd=1
    while notEnd:
        try:
            driver.find_elements_by_class_name('uy')[-1].click()
            sleep(2.5)
        except:
            notEnd = 0
            pass
    books_info = driver.find_elements_by_class_name('gf')
    for qual in books_info:
        books_id.append(qual.get_attribute("data-book"))
driver.quit()        
print("Quantity: ", len(books_id))

Quantity:  1774


In [4]:
class Book():
    def __init__(self):
        self.info = dict.fromkeys(["ID", "Название", "Обложка", "Возраст", "Описание",
                        "Рейтинг", "Понравилось", "В закладки", "Не понравилось", "Цена", 
                        "Серия", "Издательство", "Год", "Страниц", "Переплёт", "ISBN", 
                        "Размеры", "Формат", "Код", "В базе", "Автор", "Тематика", "Тираж"])
ages = {'ov': '0+', 'pv': '6+', 'qv': '12+', 'rv': '16+', 'sv': '18+'}        

In [5]:
def extract_book_info(book_id):
    site = requests.get("https://www.bookvoed.ru/book?id="+str(book_id)).text
    sleep(0.5)
    soap = BeautifulSoup(site, 'html.parser')
    book = Book()
    
    book.info['ID'] = book_id
    buf = soap.find('div', class_='Af')
    if buf:
        book.info['Название'] = buf.text
    buf = soap.find('img')
    if buf:
        book.info['Обложка'] = buf['src']
    for age in ages:
        if soap.find('div', class_=age+' nM'):
            book.info['Возраст'] = ages[age]
    buf = soap.find('div', class_='lw')
    if buf:
        book.info['Описание'] = buf.contents[0].strip()

    buf = soap.find('div', class_='He xe ')
    temp = buf.find('div', class_='af')
    if temp:
        book.info['Рейтинг'] = temp['style'].split()[1]
    temp = buf.find(class_='Ke Me ')
    if temp:
        book.info['Понравилось'] = temp.text.strip()
    temp = buf.find(class_='Ke Le ff')
    if temp:
        book.info['В закладки'] = temp.text.strip()
    temp = buf.find(class_='Ke Oe ')
    if temp:
        book.info['Не понравилось'] = temp.text.strip()
        
    buf = soap.find('meta', itemprop='price')
    if buf:
        book.info['Цена'] = buf['content']
    table = soap.find('table', class_='tw')
    rows = table.find_all('tr')
    data = [list(map(lambda x: x.text, row.find_all('td'))) for row in rows]
    for pare in data:
        book.info[pare[0].replace(':', '')] = pare[1]
    return book.info

In [6]:
#ОС: Windows 10
#Pool находится в вечном состоянии ожидании и отказывается запускаться
#Но насколько мне удалось вкусить многопоточное программирование и
#многозадачность, разница между управлениями потоками и процессами не велика

mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(uid):
    result = extract_book_info(uid) 
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", 
                  end='', flush=True)
    return result

with ThreadPool(10) as pool:
    result = pool.map(func_wrapper, books_id)

1770 objects are processed...

In [7]:
df = pd.DataFrame(result)
df.sort_values(by=['ID'], inplace=True)

In [8]:
with open('hw_3.csv', mode='w', encoding='utf-8-sig') as f_csv:
    df.to_csv(f_csv, index=False)